In [1]:
# Load model
import torch
import util

embeddings_path = 'glove.twitter.27B.25d.txt'
vocab_path = "./vocab.txt"
SPECIAL_TOKENS = ['<UNK>', '<PAD>', '<SOS>', '<EOS>']

MODEL = "./pretrained_model/pretrained_irony_detector.pth"
EMBEDDINGS = "./pretrained_model/pretrained_embeddings.pth"
WORD2I = "./pretrained_model/word2i.pth"
LABEL2I = "./pretrained_model/label2i.pth"

embeddings = torch.load(EMBEDDINGS)
word2i = torch.load(WORD2I)
label2i = torch.load(LABEL2I)
model = util.IronyDetector(
    input_dim=embeddings.shape[1],
    hidden_dim=64,
    embeddings_tensor=embeddings,
    pad_idx=word2i["<PAD>"],
    output_size=len(label2i),
)
model.load_state_dict(torch.load(MODEL))
model.eval()

IronyDetector(
  (embeddings): Embedding(17301, 25, padding_idx=7811)
  (dropout_layer): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(25, 64, num_layers=3, batch_first=True, dropout=0.5, bidirectional=True)
  (classifier): Linear(in_features=128, out_features=2, bias=True)
  (log_softmax): LogSoftmax(dim=2)
)

In [2]:
# Test model on vanilla
train_sentences, train_labels, test_sentences, test_labels, label2i = util.load_datasets()
test_features = util.encode_sentences(test_sentences, word2i)
test_labels = [int(l) for l in test_labels]
preds = util.predict(model, test_features)
dev_f1 = util.f1_score(preds, test_labels, label2i['1'])
dev_avg_f1 = util.avg_f1_score(preds, test_labels, set(label2i.values()))
print(f"Test F1 {dev_f1}")
print(f"Avg Test F1 {dev_avg_f1}")